## Kaggle: M5_Accuracy Competition _ Predictive Model Baseline

Competition Link: https://www.kaggle.com/c/m5-forecasting-accuracy

After preprocessing the competition files using the preprocessing pipeline notebook, locate the training, test, validation files and run all.

Training file has approximately 65 million rows, and model uses 59 columns for training.

Features are standard features, and besides encoding the categorical variables, no features are engineered. 

MSE = 0.7

For prediction:



### import libraries, adjust the setting

In [1]:
import pandas as pd
import numpy as np
import time
x=time.time()

### Adjust settings

In [2]:
#set precision to 3 for easier reading
np.set_printoptions(precision=3, suppress=True)

#set the number of columns and rows visible with pandas dataframes
pd.set_option('max_column', None)
pd.set_option('max_rows', 999)

### 2. Save Schema and Training columns

In [3]:
#schema of the final processed file
schema = {'date':'O','id':'O', 'item_id':'O', 'dept_id':'O', 'cat_id':'O',
 'store_id':'O', 'state_id':'O', 'd':'O', 'weekday':'O', 'event_name_1':'O',
 'event_type_1':'O', 'event_name_2':'O', 'event_type_2':'O', 'day':'int16',
 'wm_yr_wk':'int16', 'year':'int16', 'month':'int8', 'wday':'int8', 'snap_CA':'int8',
 'snap_TX':'int8', 'snap_WI':'int8', 'Eid al-Fitr':'int8', 'Pesach End':'int8',
 "Father's day":'int8', 'ValentinesDay':'int8', 'NBAFinalsStart':'int8', 'NewYear':'int8',
 'Chanukah End':'int8', 'StPatricksDay':'int8', 'LentWeek2':'int8', 'Cinco De Mayo':'int8',
 'Christmas':'int8', "Mother's day":'int8', 'ColumbusDay':'int8', 'VeteransDay':'int8',
 'IndependenceDay':'int8', 'SuperBowl':'int8', 'Easter':'int8', 'Halloween':'int8',
 'MartinLutherKingDay':'int8', 'OrthodoxChristmas':'int8', 'Thanksgiving':'int8', 'OrthodoxEaster':'int8',
 'EidAlAdha':'int8', 'NBAFinalsEnd':'int8', 'PresidentsDay':'int8', 'LentStart':'int8',
 'Ramadan starts':'int8', 'Purim End':'int8', 'LaborDay':'int8', 'MemorialDay':'int8',
 'Cultural':'int8', 'National':'int8', 'Religious':'int8', 'Sporting':'int8', 'FOODS_2':'int8',
 'FOODS_3':'int8', 'HOBBIES_1':'int8', 'HOBBIES_2':'int8', 'HOUSEHOLD_1':'int8',
 'HOUSEHOLD_2':'int8', 'HOBBIES':'int8', 'HOUSEHOLD':'int8', 'TX':'int8', 'WI':'int8',
 'CA_2':'int8', 'CA_3':'int8', 'CA_4':'int8', 'TX_1':'int8', 'TX_2':'int8',
 'TX_3':'int8', 'WI_1':'int8', 'WI_2':'int8', 'WI_3':'int8', 'sell_price':'float32',
 'quantity_sold':'int32'}

In [4]:
training_cols = ['id','item_id','day', 'month','wday','CA_2', 'CA_3', 'CA_4', 
                 'Chanukah End', 'Christmas', 'Cinco De Mayo', 'ColumbusDay', 
                 'Cultural', 'Easter', 'Eid al-Fitr', 'EidAlAdha', 'FOODS_2', 
                 'FOODS_3', "Father's day", 'HOBBIES', 'HOBBIES_1', 'HOBBIES_2',
                 'HOUSEHOLD', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'Halloween', 'IndependenceDay',
                 'LaborDay', 'LentStart', 'LentWeek2', 'MartinLutherKingDay', 'MemorialDay',
                 "Mother's day", 'NBAFinalsEnd', 'NBAFinalsStart', 'National', 
                 'NewYear', 'OrthodoxChristmas', 'OrthodoxEaster', 'Pesach End', 
                 'PresidentsDay', 'Purim End', 'Ramadan starts', 'Religious', 'Sporting',
                 'StPatricksDay', 'SuperBowl', 'TX', 'TX_1', 'TX_2', 'TX_3', 'Thanksgiving',
                 'ValentinesDay', 'VeteransDay', 'WI', 'WI_1', 'WI_2', 'WI_3', 
                 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'quantity_sold']

### identify the input and target columns

In [5]:
#Columns to use for training
input_cols = ['month', 'wday', 'snap_CA', 'snap_TX', 'snap_WI',
       'Eid al-Fitr', 'Pesach End', 'Father\'s day', 'ValentinesDay',
       'NBAFinalsStart', 'NewYear', 'Chanukah End', 'StPatricksDay',
       'LentWeek2', 'Cinco De Mayo', 'Christmas', 'Mother\'s day',
       'ColumbusDay', 'VeteransDay', 'IndependenceDay', 'SuperBowl', 'Easter',
       'Halloween', 'MartinLutherKingDay', 'OrthodoxChristmas', 'Thanksgiving',
       'OrthodoxEaster', 'EidAlAdha', 'NBAFinalsEnd', 'PresidentsDay',
       'LentStart', 'Ramadan starts', 'Purim End', 'LaborDay', 'MemorialDay',
       'Cultural', 'National', 'Religious', 'Sporting', 'FOODS_2', 'FOODS_3',
       'HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'HOBBIES',
       'HOUSEHOLD', 'TX', 'WI', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3',
       'WI_1', 'WI_2', 'WI_3', 'sell_price']

target_col = 'quantity_sold'

In [6]:
#save the submission file column names to a variable
submission_columns = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']

## Import the data 

In [7]:
training_file_path = '../../../../Documents/KAGGLE/WALMART/m5_training_df.csv'

validation_file_path = '../../../../Documents/KAGGLE/WALMART/v6_days_1913_1941_testing_data.csv'

testing_file_path = '../../../../Documents/KAGGLE/WALMART/m5_testing_df.csv'

In [8]:
#import the training file
training_df = pd.read_csv(training_file_path, index_col=0, dtype=schema, usecols=training_cols)

In [9]:
#import the validation file
validation_df = pd.read_csv(validation_file_path, usecols=training_cols, dtype=schema)

In [10]:
#import the testing file
testing_df = pd.read_csv(testing_file_path, usecols=training_cols, dtype=schema)

### Set up the file locations and names

In [11]:
file_name = input('Enter a name for the results file?: ')
file_path = '../../../../Documents/KAGGLE/WALMART/'
file_name = file_path+file_name

Enter a name for the results file?: 6.Multiple_Linear_Regression_Baseline_2


## Build the model

In [12]:
#create dataframes to save the results into a new dataframe

weight_dict = {}  #ths will store the best weights for each item

#make a list of the unique item_ids
items = training_df.item_id.unique()

for item in items:
    
    
    temp_df = training_df[training_df['item_id'] == item]

    X = temp_df[input_cols]   #filter the dataset on the item_id

    target = np.array(temp_df[target_col]).reshape(-1,1)   #labels



    #initialize coefficients and the bias
    np.random.seed(1)
    W = np.random.rand(59, 1)/10
    B = np.random.random(size=(1))/10
    N = np.random.rand(X.shape[0],1)/10

    #Set parameters
    epochs = 30
    early_stop_threshold = 3
    learning_rate = 0.001
    num_observations = X.shape[0]

    #initialize the loss and and a loss list to help with early stopping
    losses = [0]
    loss_score = 0


    for epoch in range(epochs):

        Y =  X.dot(W) +  B + N  #calculate the targets

        deltas = target - Y   #Find the error

        loss = deltas.pow(2).mean()   #Loss function

        

        #set up an early stopping mechanism
        #add the loss to the losses list
        losses.append(loss)
        
        if loss[0] > loss_score:

            loss_score += 1   #keep track of the change in loss function


        print(f'Epoch: {epoch}  Loss: {loss[0]:.3f}')


        if loss_score > early_stop_threshold or epoch == (epochs-1):   #If the loss is increasing 10 times or finished epochs stop learning

            weight_dict[item] = (W,B)
            
            
            #prepare a submission file with the validation file
            
            result = validation_df[validation_df['item_id'] == item][['id', 'item_id', 'day', 'quantity_sold']]   #create a new dataframe to save predictions

            result['predictions_test'] =  validation_df[validation_df.item_id==item][input_cols].dot(weight_dict[item][0])+weight_dict[item][1]#save the predictions

            result.to_csv(file_name + '_validation.csv', mode='a', header=False)
            
            
            #prepare a submission file with the ground truth file
            result = testing_df[testing_df['item_id'] == item][['id', 'item_id', 'day', 'quantity_sold']]   #create a new dataframe to save predictions

            result['predictions_test'] =  testing_df[testing_df.item_id==item][input_cols].dot(weight_dict[item][0])+weight_dict[item][1]#save the predictions

            result.to_csv(file_name + '_test.csv', mode='a', header=False)
          

            break



        #scale the deltas
        deltas_scaled = deltas / num_observations

        #Optimize
        #update the weights biases
        W = W -learning_rate * np.dot(X.T, deltas_scaled)
        B = B - learning_rate * np.sum(deltas_scaled)
        
        


Epoch: 0  Loss: 66.311
Epoch: 1  Loss: 67.983
Epoch: 2  Loss: 69.898
Epoch: 3  Loss: 72.092
Epoch: 0  Loss: 3.165
Epoch: 1  Loss: 3.171
Epoch: 2  Loss: 3.176
Epoch: 3  Loss: 3.183
Epoch: 0  Loss: 0.992
Epoch: 1  Loss: 1.027
Epoch: 2  Loss: 1.068
Epoch: 3  Loss: 1.115
Epoch: 4  Loss: 1.170
Epoch: 5  Loss: 1.234
Epoch: 6  Loss: 1.308
Epoch: 7  Loss: 1.394
Epoch: 8  Loss: 1.494
Epoch: 9  Loss: 1.610
Epoch: 10  Loss: 1.745
Epoch: 11  Loss: 1.901
Epoch: 12  Loss: 2.083
Epoch: 13  Loss: 2.294
Epoch: 14  Loss: 2.539
Epoch: 15  Loss: 2.823
Epoch: 16  Loss: 3.153
Epoch: 0  Loss: 1.284
Epoch: 1  Loss: 1.463
Epoch: 2  Loss: 1.682
Epoch: 3  Loss: 1.952
Epoch: 4  Loss: 2.282
Epoch: 5  Loss: 2.688
Epoch: 6  Loss: 3.187
Epoch: 0  Loss: 57.788
Epoch: 1  Loss: 59.201
Epoch: 2  Loss: 60.819
Epoch: 3  Loss: 62.672
Epoch: 0  Loss: 33.719
Epoch: 1  Loss: 34.300
Epoch: 2  Loss: 34.967
Epoch: 3  Loss: 35.730
Epoch: 0  Loss: 1.351
Epoch: 1  Loss: 1.563
Epoch: 2  Loss: 1.827
Epoch: 3  Loss: 2.156
Epoch: 4  Los

Epoch: 0  Loss: 1.443
Epoch: 1  Loss: 1.518
Epoch: 2  Loss: 1.605
Epoch: 3  Loss: 1.706
Epoch: 4  Loss: 1.824
Epoch: 5  Loss: 1.961
Epoch: 6  Loss: 2.120
Epoch: 7  Loss: 2.305
Epoch: 8  Loss: 2.520
Epoch: 9  Loss: 2.771
Epoch: 10  Loss: 3.062
Epoch: 0  Loss: 2.207
Epoch: 1  Loss: 3.135
Epoch: 2  Loss: 4.499
Epoch: 3  Loss: 6.503
Epoch: 0  Loss: 1.702
Epoch: 1  Loss: 1.743
Epoch: 2  Loss: 1.790
Epoch: 3  Loss: 1.844
Epoch: 4  Loss: 1.906
Epoch: 5  Loss: 1.976
Epoch: 6  Loss: 2.058
Epoch: 7  Loss: 2.151
Epoch: 8  Loss: 2.258
Epoch: 9  Loss: 2.380
Epoch: 10  Loss: 2.521
Epoch: 11  Loss: 2.682
Epoch: 12  Loss: 2.867
Epoch: 13  Loss: 3.079
Epoch: 0  Loss: 1.554
Epoch: 1  Loss: 1.583
Epoch: 2  Loss: 1.616
Epoch: 3  Loss: 1.655
Epoch: 4  Loss: 1.700
Epoch: 5  Loss: 1.752
Epoch: 6  Loss: 1.812
Epoch: 7  Loss: 1.882
Epoch: 8  Loss: 1.962
Epoch: 9  Loss: 2.056
Epoch: 10  Loss: 2.163
Epoch: 11  Loss: 2.289
Epoch: 12  Loss: 2.433
Epoch: 13  Loss: 2.601
Epoch: 14  Loss: 2.795
Epoch: 15  Loss: 3.019

Epoch: 11  Loss: 3.188
Epoch: 0  Loss: 59.875
Epoch: 1  Loss: 61.417
Epoch: 2  Loss: 63.183
Epoch: 3  Loss: 65.205
Epoch: 0  Loss: 47.869
Epoch: 1  Loss: 49.082
Epoch: 2  Loss: 50.471
Epoch: 3  Loss: 52.060
Epoch: 0  Loss: 0.840
Epoch: 1  Loss: 0.918
Epoch: 2  Loss: 1.007
Epoch: 3  Loss: 1.108
Epoch: 4  Loss: 1.225
Epoch: 5  Loss: 1.359
Epoch: 6  Loss: 1.513
Epoch: 7  Loss: 1.689
Epoch: 8  Loss: 1.891
Epoch: 9  Loss: 2.123
Epoch: 10  Loss: 2.389
Epoch: 11  Loss: 2.694
Epoch: 12  Loss: 3.043
Epoch: 0  Loss: 21.189
Epoch: 1  Loss: 21.297
Epoch: 2  Loss: 21.421
Epoch: 3  Loss: 21.564
Epoch: 0  Loss: 1.290
Epoch: 1  Loss: 1.513
Epoch: 2  Loss: 1.779
Epoch: 3  Loss: 2.098
Epoch: 4  Loss: 2.478
Epoch: 5  Loss: 2.933
Epoch: 6  Loss: 3.477
Epoch: 0  Loss: 83.117
Epoch: 1  Loss: 85.075
Epoch: 2  Loss: 87.317
Epoch: 3  Loss: 89.883
Epoch: 0  Loss: 2.173
Epoch: 1  Loss: 3.070
Epoch: 2  Loss: 4.387
Epoch: 3  Loss: 6.321
Epoch: 0  Loss: 1.322
Epoch: 1  Loss: 1.529
Epoch: 2  Loss: 1.792
Epoch: 3  Lo

Epoch: 0  Loss: 62.293
Epoch: 1  Loss: 64.024
Epoch: 2  Loss: 66.004
Epoch: 3  Loss: 68.271
Epoch: 0  Loss: 105.026
Epoch: 1  Loss: 108.545
Epoch: 2  Loss: 112.573
Epoch: 3  Loss: 117.185
Epoch: 0  Loss: 1.522
Epoch: 1  Loss: 1.592
Epoch: 2  Loss: 1.679
Epoch: 3  Loss: 1.786
Epoch: 4  Loss: 1.918
Epoch: 5  Loss: 2.080
Epoch: 6  Loss: 2.281
Epoch: 7  Loss: 2.528
Epoch: 8  Loss: 2.832
Epoch: 9  Loss: 3.208
Epoch: 0  Loss: 1.821
Epoch: 1  Loss: 2.375
Epoch: 2  Loss: 3.119
Epoch: 3  Loss: 4.118
Epoch: 0  Loss: 2.560
Epoch: 1  Loss: 2.561
Epoch: 2  Loss: 2.562
Epoch: 3  Loss: 2.563
Epoch: 4  Loss: 2.564
Epoch: 5  Loss: 2.565
Epoch: 6  Loss: 2.566
Epoch: 7  Loss: 2.568
Epoch: 8  Loss: 2.569
Epoch: 9  Loss: 2.571
Epoch: 10  Loss: 2.573
Epoch: 11  Loss: 2.576
Epoch: 12  Loss: 2.578
Epoch: 13  Loss: 2.582
Epoch: 14  Loss: 2.585
Epoch: 15  Loss: 2.589
Epoch: 16  Loss: 2.594
Epoch: 17  Loss: 2.599
Epoch: 18  Loss: 2.605
Epoch: 19  Loss: 2.612
Epoch: 20  Loss: 2.620
Epoch: 21  Loss: 2.629
Epoch: 2

Epoch: 0  Loss: 2.141
Epoch: 1  Loss: 2.147
Epoch: 2  Loss: 2.153
Epoch: 3  Loss: 2.160
Epoch: 4  Loss: 2.169
Epoch: 5  Loss: 2.178
Epoch: 6  Loss: 2.189
Epoch: 7  Loss: 2.201
Epoch: 8  Loss: 2.215
Epoch: 9  Loss: 2.231
Epoch: 10  Loss: 2.250
Epoch: 11  Loss: 2.271
Epoch: 12  Loss: 2.295
Epoch: 13  Loss: 2.323
Epoch: 14  Loss: 2.355
Epoch: 15  Loss: 2.392
Epoch: 16  Loss: 2.434
Epoch: 17  Loss: 2.482
Epoch: 18  Loss: 2.537
Epoch: 19  Loss: 2.601
Epoch: 20  Loss: 2.673
Epoch: 21  Loss: 2.756
Epoch: 22  Loss: 2.852
Epoch: 23  Loss: 2.961
Epoch: 24  Loss: 3.086
Epoch: 0  Loss: 1.197
Epoch: 1  Loss: 1.383
Epoch: 2  Loss: 1.605
Epoch: 3  Loss: 1.869
Epoch: 4  Loss: 2.184
Epoch: 5  Loss: 2.560
Epoch: 6  Loss: 3.008
Epoch: 0  Loss: 0.897
Epoch: 1  Loss: 0.984
Epoch: 2  Loss: 1.083
Epoch: 3  Loss: 1.198
Epoch: 4  Loss: 1.330
Epoch: 5  Loss: 1.482
Epoch: 6  Loss: 1.656
Epoch: 7  Loss: 1.858
Epoch: 8  Loss: 2.089
Epoch: 9  Loss: 2.355
Epoch: 10  Loss: 2.662
Epoch: 11  Loss: 3.015
Epoch: 0  Loss:

...

Epoch: 0  Loss: 1.379
Epoch: 1  Loss: 1.423
Epoch: 2  Loss: 1.476
Epoch: 3  Loss: 1.540
Epoch: 4  Loss: 1.616
Epoch: 5  Loss: 1.708
Epoch: 6  Loss: 1.819
Epoch: 7  Loss: 1.952
Epoch: 8  Loss: 2.111
Epoch: 9  Loss: 2.303
Epoch: 10  Loss: 2.533
Epoch: 11  Loss: 2.810
Epoch: 12  Loss: 3.142
Epoch: 0  Loss: 1.145
Epoch: 1  Loss: 1.246
Epoch: 2  Loss: 1.368
Epoch: 3  Loss: 1.513
Epoch: 4  Loss: 1.689
Epoch: 5  Loss: 1.899
Epoch: 6  Loss: 2.152
Epoch: 7  Loss: 2.456
Epoch: 8  Loss: 2.822
Epoch: 9  Loss: 3.261
Epoch: 0  Loss: 1.405
Epoch: 1  Loss: 1.608
Epoch: 2  Loss: 1.859
Epoch: 3  Loss: 2.170
Epoch: 4  Loss: 2.553
Epoch: 5  Loss: 3.028
Epoch: 0  Loss: 2.159
Epoch: 1  Loss: 2.195
Epoch: 2  Loss: 2.237
Epoch: 3  Loss: 2.285
Epoch: 4  Loss: 2.342
Epoch: 5  Loss: 2.407
Epoch: 6  Loss: 2.483
Epoch: 7  Loss: 2.571
Epoch: 8  Loss: 2.674
Epoch: 9  Loss: 2.793
Epoch: 10  Loss: 2.931
Epoch: 11  Loss: 3.091
Epoch: 0  Loss: 1.042
Epoch: 1  Loss: 1.158
Epoch: 2  Loss: 1.296
Epoch: 3  Loss: 1.459
Epoch

Epoch: 0  Loss: 2.359
Epoch: 1  Loss: 3.360
Epoch: 2  Loss: 4.830
Epoch: 3  Loss: 6.985
Epoch: 0  Loss: 3.217
Epoch: 1  Loss: 3.241
Epoch: 2  Loss: 3.268
Epoch: 3  Loss: 3.300
Epoch: 0  Loss: 0.959
Epoch: 1  Loss: 1.033
Epoch: 2  Loss: 1.119
Epoch: 3  Loss: 1.220
Epoch: 4  Loss: 1.337
Epoch: 5  Loss: 1.474
Epoch: 6  Loss: 1.634
Epoch: 7  Loss: 1.820
Epoch: 8  Loss: 2.037
Epoch: 9  Loss: 2.289
Epoch: 10  Loss: 2.584
Epoch: 11  Loss: 2.927
Epoch: 12  Loss: 3.328
Epoch: 0  Loss: 1.165
Epoch: 1  Loss: 1.342
Epoch: 2  Loss: 1.553
Epoch: 3  Loss: 1.806
Epoch: 4  Loss: 2.108
Epoch: 5  Loss: 2.469
Epoch: 6  Loss: 2.900
Epoch: 7  Loss: 3.415
Epoch: 0  Loss: 1.977
Epoch: 1  Loss: 2.143
Epoch: 2  Loss: 2.361
Epoch: 3  Loss: 2.648
Epoch: 4  Loss: 3.023
Epoch: 0  Loss: 1.492
Epoch: 1  Loss: 1.581
Epoch: 2  Loss: 1.685
Epoch: 3  Loss: 1.808
Epoch: 4  Loss: 1.952
Epoch: 5  Loss: 2.122
Epoch: 6  Loss: 2.322
Epoch: 7  Loss: 2.558
Epoch: 8  Loss: 2.836
Epoch: 9  Loss: 3.163
Epoch: 0  Loss: 1.676
Epoch: 

Epoch: 0  Loss: 0.998
Epoch: 1  Loss: 1.114
Epoch: 2  Loss: 1.251
Epoch: 3  Loss: 1.414
Epoch: 4  Loss: 1.607
Epoch: 5  Loss: 1.835
Epoch: 6  Loss: 2.104
Epoch: 7  Loss: 2.424
Epoch: 8  Loss: 2.801
Epoch: 9  Loss: 3.249
Epoch: 0  Loss: 7.625
Epoch: 1  Loss: 7.867
Epoch: 2  Loss: 8.152
Epoch: 3  Loss: 8.487
Epoch: 0  Loss: 1.078
Epoch: 1  Loss: 1.154
Epoch: 2  Loss: 1.245
Epoch: 3  Loss: 1.354
Epoch: 4  Loss: 1.486
Epoch: 5  Loss: 1.645
Epoch: 6  Loss: 1.837
Epoch: 7  Loss: 2.068
Epoch: 8  Loss: 2.346
Epoch: 9  Loss: 2.681
Epoch: 10  Loss: 3.085
Epoch: 0  Loss: 2.340
Epoch: 1  Loss: 2.342
Epoch: 2  Loss: 2.344
Epoch: 3  Loss: 2.347
Epoch: 4  Loss: 2.351
Epoch: 5  Loss: 2.354
Epoch: 6  Loss: 2.359
Epoch: 7  Loss: 2.364
Epoch: 8  Loss: 2.369
Epoch: 9  Loss: 2.376
Epoch: 10  Loss: 2.384
Epoch: 11  Loss: 2.392
Epoch: 12  Loss: 2.403
Epoch: 13  Loss: 2.415
Epoch: 14  Loss: 2.429
Epoch: 15  Loss: 2.445
Epoch: 16  Loss: 2.464
Epoch: 17  Loss: 2.486
Epoch: 18  Loss: 2.513
Epoch: 19  Loss: 2.543

Epoch: 26  Loss: 2.265
Epoch: 27  Loss: 2.287
Epoch: 28  Loss: 2.312
Epoch: 29  Loss: 2.341
Epoch: 0  Loss: 1.398
Epoch: 1  Loss: 1.666
Epoch: 2  Loss: 1.997
Epoch: 3  Loss: 2.406
Epoch: 4  Loss: 2.912
Epoch: 5  Loss: 3.537
Epoch: 0  Loss: 5.258
Epoch: 1  Loss: 5.306
Epoch: 2  Loss: 5.361
Epoch: 3  Loss: 5.424
Epoch: 0  Loss: 1.462
Epoch: 1  Loss: 1.508
Epoch: 2  Loss: 1.562
Epoch: 3  Loss: 1.624
Epoch: 4  Loss: 1.695
Epoch: 5  Loss: 1.778
Epoch: 6  Loss: 1.873
Epoch: 7  Loss: 1.983
Epoch: 8  Loss: 2.109
Epoch: 9  Loss: 2.255
Epoch: 10  Loss: 2.424
Epoch: 11  Loss: 2.618
Epoch: 12  Loss: 2.842
Epoch: 13  Loss: 3.100
Epoch: 0  Loss: 1.116
Epoch: 1  Loss: 1.213
Epoch: 2  Loss: 1.325
Epoch: 3  Loss: 1.456
Epoch: 4  Loss: 1.608
Epoch: 5  Loss: 1.785
Epoch: 6  Loss: 1.990
Epoch: 7  Loss: 2.229
Epoch: 8  Loss: 2.506
Epoch: 9  Loss: 2.829
Epoch: 10  Loss: 3.204
Epoch: 0  Loss: 1.491
Epoch: 1  Loss: 1.640
Epoch: 2  Loss: 1.826
Epoch: 3  Loss: 2.058
Epoch: 4  Loss: 2.349
Epoch: 5  Loss: 2.712
E

Epoch: 0  Loss: 1.795
Epoch: 1  Loss: 2.318
Epoch: 2  Loss: 3.025
Epoch: 3  Loss: 3.981
Epoch: 0  Loss: 1.575
Epoch: 1  Loss: 1.607
Epoch: 2  Loss: 1.644
Epoch: 3  Loss: 1.687
Epoch: 4  Loss: 1.737
Epoch: 5  Loss: 1.794
Epoch: 6  Loss: 1.861
Epoch: 7  Loss: 1.938
Epoch: 8  Loss: 2.027
Epoch: 9  Loss: 2.130
Epoch: 10  Loss: 2.250
Epoch: 11  Loss: 2.388
Epoch: 12  Loss: 2.549
Epoch: 13  Loss: 2.734
Epoch: 14  Loss: 2.949
Epoch: 15  Loss: 3.198
Epoch: 0  Loss: 2.551
Epoch: 1  Loss: 2.686
Epoch: 2  Loss: 2.862
Epoch: 3  Loss: 3.092
Epoch: 0  Loss: 5.841
Epoch: 1  Loss: 5.890
Epoch: 2  Loss: 5.946
Epoch: 3  Loss: 6.008
Epoch: 0  Loss: 1.091
Epoch: 1  Loss: 1.154
Epoch: 2  Loss: 1.227
Epoch: 3  Loss: 1.311
Epoch: 4  Loss: 1.408
Epoch: 5  Loss: 1.521
Epoch: 6  Loss: 1.652
Epoch: 7  Loss: 1.804
Epoch: 8  Loss: 1.980
Epoch: 9  Loss: 2.184
Epoch: 10  Loss: 2.420
Epoch: 11  Loss: 2.695
Epoch: 12  Loss: 3.013
Epoch: 0  Loss: 1.357
Epoch: 1  Loss: 1.441
Epoch: 2  Loss: 1.543
Epoch: 3  Loss: 1.665
E

### Export the results

In [13]:
submission = pd.read_csv(file_name+'_test.csv', index_col=0, header=None)
submission.columns = ['id', 'item_id', 'day', 'quantity_sold', 'predictions']
submission_file = submission.drop(['item_id','quantity_sold'], axis=1)
submission_file = submission_file.pivot_table(index=['id'], columns='day', values='predictions')
submission_file.columns = submission_columns
for col in submission_file:
    submission_file [col] = submission_file[col].apply(lambda x: x if x>0 else 0)
    
submission_file.to_csv(file_path+'m5_test_submission_ML_GD_LR-001_EStop-3_RSeed-1_Loss-MSE_Epoch-30.csv')

In [14]:
submission = pd.read_csv(file_name+'_validation.csv', index_col=0, header=None)
submission.columns = ['id', 'item_id', 'day', 'quantity_sold', 'predictions']
submission_file = submission.drop(['item_id','quantity_sold'], axis=1)
submission_file = submission_file.pivot_table(index=['id'], columns='day', values='predictions')
submission_file.columns = submission_columns
for col in submission_file:
    submission_file [col] = submission_file[col].apply(lambda x: x if x>0 else 0)
    
submission_file.to_csv(file_path+'m5_validation_submission_ML_GD_LR-001_EStop-3_RSeed-1_Loss-MSE_Epoch-30.csv')

#### print the time

In [15]:
y=time.time()

In [16]:
print(f'It took {(y-x)/60: .1f} minutes to complete the run')

It took  154.1 minutes to complete the run


### Check the output

In [18]:
#check the weights
weight_dict

{'HOBBIES_1_008': (array([[-0.033],
         [ 0.025],
         [-0.004],
         [ 0.026],
         [ 0.011],
         [ 0.009],
         [ 0.019],
         [ 0.034],
         [ 0.04 ],
         [ 0.054],
         [ 0.042],
         [ 0.068],
         [ 0.02 ],
         [ 0.088],
         [ 0.003],
         [ 0.067],
         [ 0.042],
         [ 0.056],
         [ 0.014],
         [ 0.02 ],
         [ 0.08 ],
         [ 0.097],
         [ 0.031],
         [ 0.069],
         [ 0.088],
         [ 0.089],
         [ 0.008],
         [ 0.004],
         [ 0.017],
         [ 0.088],
         [ 0.01 ],
         [ 0.042],
         [ 0.096],
         [ 0.053],
         [ 0.069],
         [ 0.031],
         [ 0.068],
         [ 0.083],
         [ 0.002],
         [ 0.075],
         [ 0.099],
         [ 0.063],
         [ 0.028],
         [ 0.079],
         [ 0.01 ],
         [ 0.033],
         [ 0.091],
         [ 0.027],
         [ 0.026],
         [ 0.012],
         [-0.001],
         [ 0.0

### Import the validation_file

In [64]:
validation_df = pd.read_csv('../../../../Documents/KAGGLE/WALMART/v6_days_1913_1941_testing_data.csv', index_col=0)

C:\Users\galip\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (34,35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\galip\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


# Make the predictions

In [ ]:
#Prepare the submission file
kaggle_output = validation_df[['id', 'item_id', 'day', 'quantity_sold']].copy()
kaggle_output['predictions'] = 0

for i in range(len(validation_df)):
    
    x = validation_df.iloc[i][input_cols]
    
    item = validation_df.loc[i, 'item_id']
    
    y = x.dot(weight_dict[item][0]) + weight_dict[item][1]
    
    kaggle_output.loc[i, 'predictions'] = y[0]
    

kaggle_output = kaggle_output.pivot_table(index='id', values='predictions', columns='day')
kaggle_output.columns = submission_columns
kaggle_output.to_csv(file_path+'kaggle_m5_validation_submission_ML_GD_LR-001_EStop-3_RSeed-1_Loss-MSE_Epoch-30.csv')
kaggle_output.head()